## Model training

In [ ]:
# Cell 1: Setup & Imports (e10)

import pandas as pd
import numpy as np
from pathlib import Path
import gc

from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error

import joblib

BASE_DIR = Path.home() / "work"
FEATURES_PATH = BASE_DIR / "features_sampled_e10_2023_2024.parquet"

TRAIN_END = pd.Timestamp("2024-06-30")
VALID_END = pd.Timestamp("2024-09-30")

print("Setup complete.")

Setup complete.


In [ ]:
# Cell 2: Data Loading, Optimisation & Time Split

# 1. Load feature parquet (with intra-day lags)
df = pd.read_parquet(FEATURES_PATH)

keep_cols = [
    "station_uuid",
    "date",
    "time_cell",
    "price",
    "price_lag_1d",
    "price_lag_2d",
    "price_lag_3d",
    "price_lag_7d",
    "price_lag_1cell",
    "price_lag_2cell",
    "price_lag_3cell",
    "price_lag_4cell",
]
df = df[keep_cols]

# Types / memory
df["date"] = pd.to_datetime(df["date"])

float_cols = [
    c
    for c in df.columns
    if c.startswith("price") or c == "time_cell"
]
for c in float_cols:
    df[c] = df[c].astype("float32")

# Deterministic ordering: by station, then date, then time_cell
df = df.sort_values(["station_uuid", "date", "time_cell"]).reset_index(drop=True)

print(f"Data loaded. Shape: {df.shape}")

# 2. Time-based split
TARGET = "price"

train_mask = df["date"] <= TRAIN_END
valid_mask = (df["date"] > TRAIN_END) & (df["date"] <= VALID_END)
test_mask  = df["date"] > VALID_END

print("Rows per split:")
print(f"Train: {train_mask.sum():,}")
print(f"Valid: {valid_mask.sum():,}")
print(f"Test : {test_mask.sum():,}")

feature_cols = [
    "price_lag_1d",
    "price_lag_2d",
    "price_lag_3d",
    "price_lag_7d",
    "price_lag_1cell",
    "price_lag_2cell",
    "price_lag_3cell",
    "price_lag_4cell",
]

X = df[feature_cols]
y = df[TARGET]

X_train = X[train_mask]
y_train = y[train_mask]

X_valid = X[valid_mask]
y_valid = y[valid_mask]

X_test  = X[test_mask]
y_test  = y[test_mask]

# Keep df out of memory from now on
del df, X, y
gc.collect()

print("Split complete.")

Data loaded. Shape: (13597460, 12)
Rows per split:
Train: 10,149,078
Valid: 1,730,323
Test : 1,718,059
Split complete.


In [3]:
# Cell 3: Simple Baselines on TEST set

def get_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    return mae, rmse

print("--- BASELINES (TEST SET) ---")

b1_mae, b1_rmse = get_metrics(y_test, X_test["price_lag_1d"])
print(f"Yesterday (Lag1):  MAE={b1_mae:.4f} | RMSE={b1_rmse:.4f}")

b2_mae, b2_rmse = get_metrics(y_test, X_test["price_lag_7d"])
print(f"Last Week (Lag7):  MAE={b2_mae:.4f} | RMSE={b2_rmse:.4f}")

avg_3d = (
    X_test["price_lag_1d"]
    + X_test["price_lag_2d"]
    + X_test["price_lag_3d"]
) / 3
b3_mae, b3_rmse = get_metrics(y_test, avg_3d)
print(f"Avg 3 Days:        MAE={b3_mae:.4f} | RMSE={b3_rmse:.4f}")

--- BASELINES (TEST SET) ---
Yesterday (Lag1):  MAE=0.0142 | RMSE=0.0242
Last Week (Lag7):  MAE=0.0208 | RMSE=0.0295
Avg 3 Days:        MAE=0.0143 | RMSE=0.0218


In [ ]:
# Cell 4: Train 5 horizon-specific e10 models and export

# --- Model configurations ---

MODEL_CONFIGS = [
    {
        "name": "e10_h0_daily",
        "description": "H>=5 cells ahead – only daily lags",
        "features": ["price_lag_1d", "price_lag_2d", "price_lag_3d", "price_lag_7d"],
        "filename": "fuel_price_model_ARDL_e10_h0_daily.joblib",
    },
    {
        "name": "e10_h1_1cell",
        "description": "H=1 cell ahead – daily lags + today price 1 cell before target",
        "features": [
            "price_lag_1d",
            "price_lag_2d",
            "price_lag_3d",
            "price_lag_7d",
            "price_lag_1cell",
        ],
        "filename": "fuel_price_model_ARDL_e10_h1_1cell.joblib",
    },
    {
        "name": "e10_h2_2cell",
        "description": "H=2 cells ahead – daily lags + today price 2 cells before target",
        "features": [
            "price_lag_1d",
            "price_lag_2d",
            "price_lag_3d",
            "price_lag_7d",
            "price_lag_2cell",
        ],
        "filename": "fuel_price_model_ARDL_e10_h2_2cell.joblib",
    },
    {
        "name": "e10_h3_3cell",
        "description": "H=3 cells ahead – daily lags + today price 3 cells before target",
        "features": [
            "price_lag_1d",
            "price_lag_2d",
            "price_lag_3d",
            "price_lag_7d",
            "price_lag_3cell",
        ],
        "filename": "fuel_price_model_ARDL_e10_h3_3cell.joblib",
    },
    {
        "name": "e10_h4_4cell",
        "description": "H=4 cells ahead – daily lags + today price 4 cells before target",
        "features": [
            "price_lag_1d",
            "price_lag_2d",
            "price_lag_3d",
            "price_lag_7d",
            "price_lag_4cell",
        ],
        "filename": "fuel_price_model_ARDL_e10_h4_4cell.joblib",
    },
]

results = []

for cfg in MODEL_CONFIGS:
    name = cfg["name"]
    feats = cfg["features"]
    filename = cfg["filename"]

    print("\n" + "=" * 80)
    print(f"Training model: {name}")
    print(f"Features: {feats}")

    # Filter out rows where any required feature is missing (intra-day lags at day start)
    train_rows = X_train[feats].notna().all(axis=1)
    valid_rows = X_valid[feats].notna().all(axis=1)
    test_rows  = X_test[feats].notna().all(axis=1)

    Xtr, ytr = X_train.loc[train_rows, feats], y_train.loc[train_rows]
    Xv,  yv  = X_valid.loc[valid_rows, feats], y_valid.loc[valid_rows]
    Xte, yte = X_test.loc[test_rows,  feats], y_test.loc[test_rows]

    print(f"Train rows: {len(Xtr):,} | Valid rows: {len(Xv):,} | Test rows: {len(Xte):,}")

    # Define pipeline: Standardise -> Ridge regression
    model = Pipeline(
        [
            ("scaler", StandardScaler()),
            ("regressor", Ridge(alpha=1.0)),
        ]
    )

    # Fit
    model.fit(Xtr, ytr)

    # Validation metrics
    yv_pred = model.predict(Xv)
    val_mae = mean_absolute_error(yv, yv_pred)
    val_rmse = np.sqrt(mean_squared_error(yv, yv_pred))

    # Test metrics
    yte_pred = model.predict(Xte)
    test_mae = mean_absolute_error(yte, yte_pred)
    test_rmse = np.sqrt(mean_squared_error(yte, yte_pred))

    print(f"[{name}] VALID  MAE={val_mae:.4f} | RMSE={val_rmse:.4f}")
    print(f"[{name}] TEST   MAE={test_mae:.4f} | RMSE={test_rmse:.4f}")

    # Relative improvement vs Lag1 baseline on the (filtered) test subset
    b1_mae_sub, b1_rmse_sub = get_metrics(yte, X_test.loc[test_rows, "price_lag_1d"])
    impr_mae = ((b1_mae_sub - test_mae) / b1_mae_sub) * 100.0
    print(f"[{name}] Improvement vs Lag1 baseline (MAE): {impr_mae:.2f}%")

    results.append(
        {
            "name": name,
            "features": feats,
            "val_mae": val_mae,
            "val_rmse": val_rmse,
            "test_mae": test_mae,
            "test_rmse": test_rmse,
            "improvement_vs_lag1_mae_pct": impr_mae,
        }
    )

    # Export model
    export_path = BASE_DIR / filename
    joblib.dump(model, export_path)
    print(f"Model saved to {export_path}")

print("\nSUMMARY OF ALL MODELS:")
for r in results:
    print(
        f"{r['name']:18s}  "
        f"Test MAE={r['test_mae']:.4f}  RMSE={r['test_rmse']:.4f}  "
        f"ΔMAE vs Lag1={r['improvement_vs_lag1_mae_pct']:.2f}%"
    )


Training model: e10_h0_daily
Features: ['price_lag_1d', 'price_lag_2d', 'price_lag_3d', 'price_lag_7d']
Train rows: 10,149,078 | Valid rows: 1,730,323 | Test rows: 1,718,059
[e10_h0_daily] VALID  MAE=0.0143 | RMSE=0.0207
[e10_h0_daily] TEST   MAE=0.0139 | RMSE=0.0208
[e10_h0_daily] Improvement vs Lag1 baseline (MAE): 2.32%
Model saved to /home/jovyan/work/fuel_price_model_ARDL_e10_h0_daily.joblib

Training model: e10_h1_1cell
Features: ['price_lag_1d', 'price_lag_2d', 'price_lag_3d', 'price_lag_7d', 'price_lag_1cell']
Train rows: 9,937,928 | Valid rows: 1,694,275 | Test rows: 1,682,210
[e10_h1_1cell] VALID  MAE=0.0109 | RMSE=0.0164
[e10_h1_1cell] TEST   MAE=0.0111 | RMSE=0.0170
[e10_h1_1cell] Improvement vs Lag1 baseline (MAE): 22.56%
Model saved to /home/jovyan/work/fuel_price_model_ARDL_e10_h1_1cell.joblib

Training model: e10_h2_2cell
Features: ['price_lag_1d', 'price_lag_2d', 'price_lag_3d', 'price_lag_7d', 'price_lag_2cell']
Train rows: 9,726,778 | Valid rows: 1,658,227 | Test ro